In [56]:
import pickle
import os 
df = pickle.load(open(os.path.join('./datasets/flow/Medboit_all_seq_30_2.pickle'), 'rb'))
df.shape

(9993, 30)

In [57]:
import os

[1606540174.193376,
 1606540174.194401,
 1606540174.195664,
 1606540174.196348,
 1606540174.196902,
 1606540174.197497,
 1606540174.198002,
 1606540174.198846,
 1606540174.199416,
 1606540174.199877,
 1606540174.200754,
 1606540174.201282,
 1606540174.201953,
 1606540174.20249,
 1606540174.202894,
 1606540174.203367,
 1606540174.203773,
 1606540174.204297,
 1606540174.204773,
 1606540174.205343,
 1606540174.205864,
 1606540174.206266,
 1606540174.206742,
 1606540174.207225,
 1606540174.207767,
 1606540174.20841,
 1606540174.208869,
 1606540174.209357,
 1606540174.209741,
 1606540174.210106,
 1606540174.210469,
 1606540174.210764,
 1606540174.211063,
 1606540174.211361,
 1606540174.307452,
 1606540174.371261,
 1606540174.37263,
 1606540174.373663,
 1606540174.374639,
 1606540174.375377,
 1606540174.376132,
 1606540174.376739,
 1606540174.377208,
 1606540174.377681,
 1606540174.378223,
 1606540174.378782,
 1606540174.379257,
 1606540174.379828,
 1606540176.116449,
 1606540176.31924,
 160